0. Error classes.

1. Pick open facilities from candidate locations. We may have an algorithm here to eliminate some possible combinations

    Inputs: 

2. Obtain an initial solution for the selected open facilities. Create balanced disttricts iteratively. Validity check for population equality.

    Inputs: graph, populations, travel times. Output: partition, assignment, cut edges, induced subgraphs.

3. Recom: selecting a pair of districts, recombining and repartitioning, validty check.  

    Inputs: partition with its outputs, 

4. Markov chain class / function

5. Run the chain. Analyse it. 

** Frozen graph?

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import plotly.express as px
import plotly.io as pio

from data_utils import assign, DataHandler, plot
from prepared_data import load
import Partition, recom, tree
import pickle
import gzip



# Load Data

In [2]:
# Load graph
handler = DataHandler()

graph = handler.load_graph()
# Load candidate facilities
candidates = handler.load_candidates()
# Load chicago geodataframe
chicago = handler.load_chicago()
# Load travel times
#travel_times = load(file_path='/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/travel_times.pkl.gz')

In [3]:
with gzip.open('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/travel_times.pkl.gz', 'rb') as file:
    travel_time=pickle.load(file)

# Initial Solution

In [4]:
k_ms = 100
#partitioned_tree, spanning_tree, clusterim, populations, total_travel, open_facilities = old.generate_initial_partition_seed(graph, candidates, travel_times, k_ms)
#initial_data = partitioned_tree, spanning_tree, clusterim, populations, total_travel, open_facilities 
#load.cache_data(initial_data, file_path='prepared_data/initial_data.pkl', method='pickle')
initial_data = handler.load_initial_data()
partitioned_tree, spanning_tree, clusterim, populations, total_travel, open_facilities = initial_data

In [5]:
assign(graph, chicago, clusterim, attribute='initial_cluster', question='is_initial_center')
#plot.districts(chicago, open_facilities, attribute='initial_cluster')

# Markov Chain

Example usage:

from functools import partial

from gerrychain import MarkovChain

from gerrychain.proposals import recom

...define constraints, accept, partition, total_steps here...

Ideal population: pop_target = sum(populations.values()) / len(partition.keys())

proposal = partial(recom, pop=populations, pop_target=pop_target, epsilon=.05, node_repeats=10)

chain = MarkovChain(proposal, constraints, accept, partition, total_steps)

In [6]:
total = sum(populations[center] for center in populations.keys())

In [7]:
average = total / 100
average

27838.91

In [8]:
0.8*average

22271.128

In [9]:
max(graph.nodes[node]['pop'] for node in graph.nodes)

4689

Issue 1. Epsilon is getting to be too small. This avoids new districts after approx 40 iterations. 

epsilon = 0.08 -> 51 districts

epsilon == 0.4 -> 55 districts

epsilon - 0.1 -> 46 districts

Question: How are we going evaluate the value of a districting?

Task 1. Make a function that finds the minimum balanced subgraph of graph with 1 center.

Task 2. Don't fix open facilities. Get a subtree with an appropriate population, no matter how many facilities it has. This solves the facility picking issue. Also, we can consider playing with number of doctor nurse teams here.

Task 3. Start using real candidate locations.

Task 4. Set epsilon very high. Imagine you remerge and resplit two districts: Make pop_target half of sum of their populations. Populations will converge to real pop_target.

Idea 1: Dynamically set candidates through the iteration. 

Idea 2: Eliminate some combinations of candidates to lower the complexity.

In [ ]:
districts = tree.recursive_partition(graph, num_centers= 100, total_pop= total, epsilon=0.08, max_attempts=100)

In [4]:
"""with gzip.open('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/result.pkl.gz', 'wb') as file:
    pickle.dump(districts, file)"""
    
handler = DataHandler()


In [ ]:
vars(handler)

In [7]:
with gzip.open('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/result.pkl.gz', 'rb') as file:
        districts = pickle.load(file)

In [11]:
handler = DataHandler()
graph = handler.load_graph()
chicago = handler.load_chicago()

In [11]:
def plot(data, centers, attribute: str, color=None, fake_center=None):
    # Ensure data[attribute] has appropriate type for indexing
    data[attribute] = pd.Categorical(data[attribute])
    
    # Map each cluster to a color using a cycle of the Plotly qualitative palette
    colors = px.colors.qualitative.Plotly  # This is an example palette
    color_map = {cat: colors[i % len(colors)] for i, cat in enumerate(data[attribute].cat.categories)}
    data['color'] = data[attribute].map(color_map)

    fig = px.choropleth_mapbox(
        data,
        geojson=data.geometry.__geo_interface__,
        locations=data.index,
        color=data['color'],
        mapbox_style="open-street-map",
        center={"lat": data.geometry.centroid.y.mean(), "lon": data.geometry.centroid.x.mean()},
        height=800,
        zoom=10,
        opacity=0.5,
        color_discrete_map="identity",  # Ensure this uses the direct mapping of assigned colors
        hover_data=[data['pop']]  # Show population data on hover
    )

    # Add cluster centers as markers
    for center in centers:
        center_point = data.loc[center].geometry.centroid
        fig.add_scattermapbox(
            lat=[center_point.y],
            lon=[center_point.x],
            mode='markers',
            marker=dict(size=10, color='black'),  # Black markers for centers
            name=f'District={center}'
        )

    return fig.show()

In [ ]:

incomplete_nodes = []
incomplete_pop = {}


for center in districts.keys():
    incomplete_nodes = incomplete_nodes + list(districts[center])
    incomplete_pop[center] = sum(graph.nodes[node]['pop'] for node in districts[center])

incomplete_subgraph = graph.subgraph(incomplete_nodes)
incomplete_centers = list(districts.keys())


"Plot Initial"

incomplete_subgraph_nodes = list(incomplete_subgraph.nodes())
incomplete_chicago = chicago.loc[incomplete_subgraph_nodes]

assign(incomplete_subgraph, incomplete_chicago, districts, attribute='initial_cluster', question='test_initial_center')
plot(incomplete_chicago, incomplete_centers, attribute='initial_cluster')



In [ ]:
import random
fake_center = random.choice(list(set(open_facilities) - set(incomplete_centers)))

districts[fake_center] = {node for node in graph.nodes if node not in incomplete_nodes}

assign(graph, chicago, districts, attribute='initial_cluster', question='test_initial_center')
plot(chicago, open_facilities, attribute='initial_cluster')

In [ ]:
"""chicago.plot(
    column="POP2010",
    legend=True,
    scheme="quantiles",
    figsize=(15, 10),
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
        "hatch": "///",
        "label": "Missing values",
    },
);"""


In [ ]:
plot(chicago_test, centers_test, attribute='final_initial_cluster')

In [ ]:
import random
node = random.choice(list(graph.nodes()))
graph.nodes[node]

In [ ]:
partition = Partition(G, assignment, districts, measures, parent, flips, flows, edge_flows, cache)
induced_subgraph = partition.get_induced_subgraph(1)  # get the induced subgraph for district 1

In [ ]:
yes=0
no=0
for node in graph.nodes:
    if graph.nodes[node].get('is_initial_center', False)==True:
        no +=1
    else:        
        yes +=1


print(yes)
print(no)
print(yes+no)